# **데이터셋 수집 방법**

**step1. LEAGUE-V4에서 특정 랭크의 소환사 닉네임을 가져온다.**

**step2. SUMMONER-V4에서 소환사 닉네임으로 puuid를 가져온다.**

**step3. MATCH-V4에서 puuid로 match id를 받아온다.**

**step4. MATCH-V4에서 match id로 인게임 정보를 받아온다.** 

**LEAGUE-V4에서 특정 랭크의 소환사 닉네임을 가져온다.**

In [1]:
import requests
import json
import time
import csv
import numpy as np
import pandas as pd

# for문 진행률 확인 라이브러리
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

**API 발급 키 수정**

**24시간마다 재발급 후 수정 필요**

In [2]:
api_key1 = 'your key' 
api_key2 = 'your key' 
api_key3 = 'your key' 
api_key4 = 'your key' 
api_key5 = 'your key' 
api_key6 = 'your key' 
api_key7 = 'your key' 
api_key8 = 'your key' 
api_key9 = 'your key' 
api_key10 = 'your key' 
api_key11 = 'your key'

api_keys  = [api_key1, api_key2, api_key3, api_key4, api_key5, api_key6,api_key7,api_key8,api_key9,api_key10]


**마스터 랭크 이상인 플레이어의 닉네임 추출**

In [ ]:

#마스터랭크
mUrl = f'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key1}'
r1 = requests.get(mUrl)
r1 = r1.json()['entries'] #소환사의 고유 id

#그랜드마스터랭크
gmUrl = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key1
r2 = requests.get(gmUrl)
r2 = r2.json()['entries'] #소환사의 고유 id

#챌린져랭크
cUrl = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key1
r3 = requests.get(cUrl)
r3 = r3.json()['entries'] #소환사의 고유 id

# print(r.json())

summonerId = []
CsummonerId = []

num = 0

for i in r1:
    summonerId.append(i['summonerName'])
    num += 1

for i in r2:
    summonerId.append(i['summonerName'])
    num += 1

for i in r3:
    CsummonerId.append(i['summonerName'])
    summonerId.append(i['summonerName'])
    num += 1

print(num)
print(summonerId)

In [ ]:
print(len(summonerId))

**추출한 플레이어 닉네임으로 puuid 추출**

In [ ]:
summonerpuuId = []

# 반복문을 사용하여 모든 데이터에 대해 api 요청
#for i, data in enumerate(tqdm(summonerId)):
for i in tqdm(range(len(summonerId))):
    # 현재 api_key와 남은 api 요청 횟수
    api_key, remaining = api_keys[i % 10], i % 1000
    # api_key의 요청 제한에 걸릴 경우 2분 휴식
    #if remaining == 999 and api_key == api_keys[9]:
    #    print('대기')
    #    time.sleep(120)
    # api 요청
    #url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{data}/ids?queue=420&type=ranked&start=0&count=5&api_key={api_key}'
    url = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summonerId[i]}?api_key={api_key}'
    response = requests.get(url)
    if response.status_code != 200:
        summonerpuuId.append('error')
        continue
    # response에서 puuid 추출하여 리스트에 저장
    summonerpuuId.append(response.json()['puuid'])

In [ ]:
print(len(summonerpuuId))
print(summonerpuuId)

**추출한 puuid를 csv파일에 저장**

In [ ]:
with open('puuid.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    for item in summonerpuuId:
        writer.writerow([item])

In [ ]:
# csv 파일 경로 설정
csv_file_path = "puuid.csv"

# csv 파일 읽기 모드로 열기
with open(csv_file_path, "r") as csvfile:

    # csv 파일 읽기 객체 생성
    csv_reader = csv.reader(csvfile)
    next(csv_reader)
    # 데이터를 저장할 리스트 생성
    data = []

    # 각 행을 읽어서 리스트에 저장
    for row in csv_reader:
        data.extend(row)

# 결과 출력
print(data)

**추출한 puuid로 matchid 추출**

In [ ]:
matchid = []
        
for i in tqdm(range(len(summonerpuuId))):
    # 현재 api_key와 남은 api 요청 횟수
    api_key, remaining = api_keys[i % 10], i % 1000
    # api_key의 요청 제한에 걸릴 경우 2분 휴식
    #if remaining == 999 and api_key == api_keys[9]:
    #    print('대기')
    #    time.sleep(120)
    # api 요청
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{summonerpuuId[i]}/ids?queue=420&start=0&count=100&api_key={api_key}'
    response = requests.get(url)
    if response.status_code != 200:
        print("파싱 에러")
        continue
    else:
        matchid += response.json()

**중복값 삭제**

In [ ]:
print(len(matchid))
matchid = list(set(matchid))
print(len(matchid))
matchid.sort()
matchid

**matchid.csv 파일 생성**

In [ ]:
with open('matchid.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    for item in matchid:
        writer.writerow([item])

**matchid.csv 파일을 data[]에 저장**

In [3]:
# csv 파일 경로 설정
csv_file_path = "matchid.csv"

# csv 파일 읽기 모드로 열기
with open(csv_file_path, "r") as csvfile:

    # csv 파일 읽기 객체 생성
    csv_reader = csv.reader(csvfile)
    next(csv_reader)
    # 데이터를 저장할 리스트 생성
    matchid = []

    # 각 행을 읽어서 리스트에 저장
    for row in csv_reader:
        matchid.extend(row)


In [4]:
data1 = matchid[80000:180000]

**데이터 범위 수정**

In [5]:
print(len(matchid))
print(len(data1))

344453
100000


**데이터프레임 정의 / 3분단위로 수집할 ms단위 시간 리스트 정의**

In [6]:
#수집할 컬럼 정의
MatchData = pd.DataFrame(columns = ['gameId','timeStamp'\
                        ,'blueWins','blueTotalGolds','blueCurrentGolds','blueTotalLevel','blueAvgLevel'\
                        ,'blueTotalMinionKills','blueTotalJungleMinionKills'
                        ,'blueFirstBlood','blueKill','blueDeath','blueAssist'\
                        ,'blueWardPlaced','blueWardKills','blueFirstTower','blueFirstInhibitor','blueFirstTowerLane'\
                        ,'blueTowerKills','blueMidTowerKills','blueTopTowerKills','blueBotTowerKills','blueInhibitor'\
                        ,'blueFirstDragon','blueDragonType','blueDragon','blueRiftHeralds','blueFirstBaron','blueBaron','blueElite'\
                        ,'redWins','redTotalGolds','redCurrentGolds','redTotalLevel','redAvgLevel'\
                        ,'redTotalMinionKills','redTotalJungleMinionKills'
                        ,'redFirstBlood','redKill','redDeath','redAssist'\
                        ,'redWardPlaced','redWardKills','redFirstTower','redFirstInhibitor','redFirstTowerLane'\
                        ,'redTowerKills','redMidTowerKills','redTopTowerKills','redBotTowerKills','redInhibitor'\
                        ,'redFirstDragon','redDragonType','redDragon','redRiftHeralds','redFirstBaron','redBaron','redElite'])

timeLine = [180000, 360000, 540000, 720000, 900000, 1080000, 1260000, 1440000, 1620000, 1800000, 1980000, 2160000, 2340000, 2520000, 2700000, 2880000, 3060000, 3240000, 3420000, 3600000]
error_list = []



# **매치 아이디로 게임데이터 3분 단위로 수집**


In [ ]:
# 데이터를 저장할 리스트 생성
data = []

for i in tqdm(range(len(data1))):
    try:
        # 현재 api_key와 남은 api 요청 횟수
        api_key = api_keys[i % 10]

        # api_key의 요청 제한에 걸릴 경우 2분 휴식
        #if remaining == 999 and api_key == api_keys[9]:
        #    print('대기')
        #    time.sleep(120)
        # api 요청
        url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{data1[i]}/timeline?api_key={api_key}'
        response = requests.get(url)
        if response.status_code != 200:
            print(response.status_code, api_key)
            print("파싱 에러")
            continue
        else:
            game_id = response.json()['metadata']['matchId']


            frames = response.json()['info']['frames']
            bw = 0
            rw = 0
            #json data에서 필요한 frames 데이터만

            framesIndex = len(frames)-1
            iii = len(frames[framesIndex]['events'])-1
            if frames[framesIndex]['events'][iii]['winningTeam'] == 100:
                bw = 1
                rw = 0
            else:
                bw = 0
                rw = 1

            lcList = [0]
            #시작하고 n분 즉, 수집하고 싶은 시간까지의 인덱스가 어디있을지 추출하는 코드
            for i in range(len(timeLine)):
                lc0 = 0 # 위치
                while True:
                    try:
                        timestamps = frames[lc0]['timestamp']

                        if timestamps <= timeLine[i]: # n Minute를 설정(Ms단위의 timeline)
                            lc0 += 1

                        else:
                            lc = lc0-1
                            lcList.append(lc)
                            break
                    except:
                        lc = lc0 - 1
                        lcList.append(lc)
                        
                        break
                    
            lcList = list(set(lcList)) #[0,5,8,15]
            lcList.sort()
            
            #timestamp별로 독립적인 변수들을 나타내므로 n분까지의 데이터를 수집하기 위해서는 계속 중첩해서 더해줘야 함
            blue_kill, red_kill = 0,0
            blue_firstkill, red_firstkill = 0,0
            blue_assist, red_assist = 0,0
            red_death, blue_death = 0,0
            blue_wardplace, red_wardplace = 0,0
            blue_wardkill, red_wardkill = 0,0
            blue_elite, red_elite = 0,0
            blue_rift, red_rift = 0,0
            blue_dragon, red_dragon = 0,0
            blue_baron, red_baron = 0,0
            blue_firstdragon, red_firstdragon = 0,0
            blue_dragontype, red_dragontype = [],[]
            blue_firstbaron, red_firstbaron = 0,0
            blue_tower,red_tower = 0,0
            blue_firsttower, red_firsttower = 0,0
            blue_firsttowerlane, red_firsttowerlane = [],[]
            blue_midtower, red_midtower = 0,0
            blue_toptower, red_toptower = 0,0
            blue_bottower, red_bottower = 0,0
            blue_inhibitor, red_inhibitor = 0,0
            blue_firstinhibitor, red_firstinhibitor = 0,0

            for lcIndex in range(len(lcList)-1):
                # participants 1~5 까지는 blueteam, 6~10까지는 redteam
                participant = frames[lcList[lcIndex+1]]['participantFrames']

                bluetotal_gold, bluecurrent_gold, bluetotal_level, \
                bluetotal_minionkill, bluetotal_jungleminionkill = [],[],[],[],[]

                redtotal_gold, redcurrent_gold, redtotal_level, \
                redtotal_minionkill, redtotal_jungleminionkill = [],[],[],[],[]


                for i in range(len(participant)):
                    i = i+1
                    if 1 <=participant[str(i)]['participantId'] <= 5:
                        bluetotal_gold.append(participant[str(i)]['totalGold'])
                        bluecurrent_gold.append(participant[str(i)]['currentGold'])
                        bluetotal_level.append(participant[str(i)]['level'])
                        bluetotal_minionkill.append(participant[str(i)]['minionsKilled'])
                        bluetotal_jungleminionkill.append(participant[str(i)]['jungleMinionsKilled'])
                    else:
                        redtotal_gold.append(participant[str(i)]['totalGold'])
                        redcurrent_gold.append(participant[str(i)]['currentGold'])
                        redtotal_level.append(participant[str(i)]['level'])
                        redtotal_minionkill.append(participant[str(i)]['minionsKilled'])
                        redtotal_jungleminionkill.append(participant[str(i)]['jungleMinionsKilled'])

                
                for y in range(lcList[lcIndex]+1,lcList[lcIndex+1]+1):

                    events = frames[y]['events']

                    for x in range(len(events)):
                        if events[x]['type'] == 'WARD_KILL':
                            if 1 <= events[x]['killerId'] <= 5:
                                blue_wardkill += 1
                            else:
                                red_wardkill += 1

                        elif events[x]['type'] == 'WARD_PLACED':
                            if 1 <= events[x]['creatorId'] <= 5:
                                blue_wardplace += 1
                            else:
                                red_wardplace += 1

                        elif events[x]['type'] == 'CHAMPION_KILL':

                            if 1 <= events[x]['killerId'] <= 5:
                                if red_kill ==0 and blue_kill ==0:
                                    blue_firstkill += 0

                                else:
                                    pass

                                blue_kill += 1
                                try:
                                    blue_assist += len(events[x]['assistingParticipantIds'])
                                except:
                                    pass
                                red_death += 1


                            else:
                                if red_kill ==0 and blue_kill ==0:
                                    red_firstkill += 0

                                else:
                                    pass

                                red_kill += 1
                                try:
                                    red_assist += len(events[x]['assistingParticipantIds'])
                                except:
                                    pass
                                blue_death += 1



                        elif events[x]['type'] == 'ELITE_MONSTER_KILL':
                            if 1 <= events[x]['killerId'] <= 5:
                                blue_elite += 1

                                if events[x]['monsterType']== 'DRAGON':
                                    if red_dragon ==0 and blue_dragon == 0:
                                         blue_firstdragon += 1
                                    else:
                                        pass

                                    blue_dragontype.append(events[x]['monsterSubType'])
                                    blue_dragon += 1



                                elif events[x]['monsterType']== 'RIFTHERALD':
                                    blue_rift += 1

                                elif events[x]['monsterType']== 'BARON_NASHOR':
                                    if red_baron ==0 and blue_baron == 0:
                                         blue_firstbaron += 1
                                    else:
                                        pass

                                    blue_baron += 1

                            else:
                                red_elite += 1

                                if events[x]['monsterType']== 'DRAGON':
                                    if red_dragon ==0 and blue_dragon == 0:
                                         red_firstdragon += 1
                                    else:
                                        pass

                                    red_dragontype.append(events[x]['monsterSubType'])
                                    red_dragon += 1



                                elif events[x]['monsterType']== 'RIFTHERALD':
                                    red_rift += 1

                                elif events[x]['monsterType']== 'BARON_NASHOR':
                                    if red_baron ==0 and blue_baron == 0:
                                         red_firstbaron += 1
                                    else:
                                        pass

                                    red_baron += 1

                        elif events[x]['type'] == 'BUILDING_KILL':
                            if 1 <= events[x]['killerId'] <= 5:

                                if events[x]['buildingType'] == 'TOWER_BUILDING':
                                    if red_tower == 0 and blue_tower ==0:
                                        blue_firsttower += 1
                                        blue_firsttowerlane.append(events[x]['laneType'])

                                    else:
                                        pass

                                    blue_tower += 1

                                    if events[x]['laneType'] == 'MID_LANE':
                                        blue_midtower += 1

                                    elif events[x]['laneType'] == 'TOP_LANE':
                                        blue_toptower += 1

                                    elif events[x]['laneType'] == 'BOT_LANE':
                                        blue_bottower += 1



                                elif events[x]['buildingType'] == 'INHIBITOR_BUILDING':
                                    if red_inhibitor == 0 and blue_inhibitor ==0:
                                        blue_firstinhibitor += 1

                                    else:
                                        pass

                                    blue_inhibitor += 1

                            else:
                                if events[x]['buildingType'] == 'TOWER_BUILDING':
                                    if red_tower == 0 and blue_tower ==0:
                                        red_firsttower += 1
                                        red_firsttowerlane.append(events[x]['laneType'])

                                    else:
                                        pass

                                    red_tower += 1

                                    if events[x]['laneType'] == 'MID_LANE':
                                        red_midtower += 1

                                    elif events[x]['laneType'] == 'TOP_LANE':
                                        red_toptower += 1

                                    elif events[x]['laneType'] == 'BOT_LANE':
                                        red_bottower += 1

                                elif events[x]['buildingType'] == 'INHIBITOR_BUILDING':
                                    if red_inhibitor == 0 and blue_inhibitor ==0:
                                        red_firstinhibitor += 1

                                    else:
                                        pass

                                    red_inhibitor += 1
                inputData = {'gameId':game_id,
                             'timeStamp':timeLine[lcIndex],
                             'blueWins':bw,
                             'blueTotalGolds':np.sum(bluetotal_gold),
                             'blueCurrentGolds':np.sum(bluecurrent_gold),
                             'blueTotalLevel':np.sum(bluetotal_level),
                             'blueAvgLevel':np.mean(bluetotal_level),
                             'blueTotalMinionKills':np.sum(bluetotal_minionkill),
                             'blueTotalJungleMinionKills':np.sum(bluetotal_jungleminionkill),
                             'blueFirstBlood':blue_firstkill,
                             'blueKill':blue_kill,
                             'blueDeath':blue_death,
                             'blueAssist':blue_assist,
                             'blueWardPlaced':blue_wardplace,
                             'blueWardKills':blue_wardkill,
                             'blueFirstTower':blue_firsttower,
                             'blueFirstInhibitor':blue_firstinhibitor,
                             'blueFirstTowerLane':blue_firsttowerlane.copy(),
                             'blueTowerKills':blue_tower,
                             'blueMidTowerKills':blue_midtower,
                             'blueTopTowerKills':blue_toptower,
                             'blueBotTowerKills':blue_bottower,
                             'blueInhibitor':blue_inhibitor,
                             'blueFirstDragon':blue_firstdragon,
                             'blueDragonType':blue_dragontype.copy(),
                             'blueDragon':blue_dragon,
                             'blueRiftHeralds':blue_rift,
                             'blueFirstBaron':blue_firstbaron,
                             'blueBaron':blue_baron,
                             'blueElite':blue_elite,
                             'redWins':rw,
                             'redTotalGolds': np.sum(redtotal_gold),
                             'redCurrentGolds': np.sum(redcurrent_gold),
                             'redTotalLevel':np.sum(redtotal_level),
                             'redAvgLevel':np.mean(redtotal_level),
                             'redTotalMinionKills': np.sum(redtotal_minionkill),
                             'redTotalJungleMinionKills':np.sum(redtotal_jungleminionkill),
                             'redFirstBlood':red_firstkill,
                             'redKill':red_kill,
                             'redDeath':red_death,
                             'redAssist':red_assist,
                             'redWardPlaced':red_wardplace,
                             'redWardKills':red_wardkill,
                             'redFirstTower':red_firsttower,
                             'redFirstInhibitor':red_firstinhibitor,
                             'redFirstTowerLane':red_firsttowerlane,
                             'redTowerKills':red_tower,
                             'redMidTowerKills':red_midtower,
                             'redTopTowerKills':red_toptower,
                             'redBotTowerKills':red_bottower,
                             'redInhibitor':red_inhibitor,
                             'redFirstDragon':red_firstdragon,
                             'redDragonType':red_dragontype.copy(),
                             'redDragon':red_dragon,
                             'redRiftHeralds':red_rift,
                             'redFirstBaron':red_firstbaron,
                             'redBaron':red_baron,
                             'redElite':red_elite}
                data.append(inputData)
                #MatchData = MatchData.append(inputData, ignore_index = True)
    except: #에러발생 시 바로 다음 반복문으로 넘어가게끔
            print('error visual')
            error_list.append(data1[i])
            pass

 15%|██████████▍                                                           | 14832/100000 [2:04:04<11:38:54,  2.03it/s]

In [ ]:
#수집할 컬럼 정의
MatchData = pd.DataFrame(data, columns = ['gameId','timeStamp'\
                        ,'blueWins','blueTotalGolds','blueCurrentGolds','blueTotalLevel','blueAvgLevel'\
                        ,'blueTotalMinionKills','blueTotalJungleMinionKills'
                        ,'blueFirstBlood','blueKill','blueDeath','blueAssist'\
                        ,'blueWardPlaced','blueWardKills','blueFirstTower','blueFirstInhibitor','blueFirstTowerLane'\
                        ,'blueTowerKills','blueMidTowerKills','blueTopTowerKills','blueBotTowerKills','blueInhibitor'\
                        ,'blueFirstDragon','blueDragonType','blueDragon','blueRiftHeralds','blueFirstBaron','blueBaron','blueElite'\
                        ,'redWins','redTotalGolds','redCurrentGolds','redTotalLevel','redAvgLevel'\
                        ,'redTotalMinionKills','redTotalJungleMinionKills'
                        ,'redFirstBlood','redKill','redDeath','redAssist'\
                        ,'redWardPlaced','redWardKills','redFirstTower','redFirstInhibitor','redFirstTowerLane'\
                        ,'redTowerKills','redMidTowerKills','redTopTowerKills','redBotTowerKills','redInhibitor'\
                        ,'redFirstDragon','redDragonType','redDragon','redRiftHeralds','redFirstBaron','redBaron','redElite'])
MatchData

In [ ]:
MatchData.to_csv("tsmd2.csv", header=True , index = False)